In [2]:
from time import sleep, time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LinearRegression
import sys
from BayesianOptimization_master.bayes_opt1 import BayesianOptimization, UtilityFunction
import statistics
from imagerecognitionmodel import main
import csv


#Initial test functions
def ackley(x, y):
    sleep(np.sqrt(x**2 + y**2) / 10)
    return -20 * np.exp(-0.2 * np.sqrt(0.5 * (x ** 2 + y ** 2))) - np.exp(0.5 * (np.cos(2 * np.pi * x) + np.cos(2 * np.pi * y))) + np.e + 20

def rosenbrock(x, y):
    sleep(np.sqrt(x**2 + y**3) / 10)
    return 100 * (y - x ** 2) ** 2 + (1 - x) ** 2

def rastrigin(x, y):
    sleep((x + y**2 + x*y)/10)
    return 20 + x ** 2 - 10 * np.cos(2 * np.pi * x) + y ** 2 - 10 * np.cos(2 * np.pi * y)

    

In [ ]:
bounds = {'num_epochs': (1, 5), 
          'batch_size': (2, 20),
          'num_fc_layers': (1, 3),
          'learning_rate': (-3.5, -1.5),
          'fc_layer_size': (32, 300),
          'kernel_size': (2,7)}

current_optimizing_fun = lambda num_epochs, batch_size, num_fc_layers, learning_rate, fc_layer_size, kernel_size: main(int(num_epochs), 
                                                                                                                        int(batch_size), 
                                                                                                                        int(num_fc_layers), 
                                                                                                                        10.0 ** learning_rate, 
                                                                                                                        int(fc_layer_size), 
                                                                                                                        int(kernel_size))

ei_max_points_time = [] 
ei_max_points = []       

eit_max_points_time = [] 
eit_max_points = []      

for i in range(30):
    ei_max_points_time.append([]) 
    ei_max_points.append([])        
    eit_max_points_time.append([]) 
    eit_max_points.append([])   

    output = []
    output.append(["EIT:"])
    eit_optimizer = BayesianOptimization(
        f=current_optimizing_fun,
        pbounds=bounds,
        allow_duplicate_points=True,
        time_travel=True
    )

    utility = UtilityFunction(kind="eit", kappa=2.5, xi=0.01)

    max_point = -1 * np.inf
    start_time = time()
    num_points_tested = 0
    while time() - start_time < 1800:
        output.append([])
        num_points_tested += 1
        earliest_t = time()
        next_point_to_probe = eit_optimizer.suggest(utility)
        print(next_point_to_probe)
        before_t = time()
        cost_to_evaluate_eit = before_t - earliest_t
        result = current_optimizing_fun(**next_point_to_probe)
        delta_t = time() - before_t
        was_best = False
        if result > max_point:
            was_best = True
            max_point = result
            eit_max_points[i].append(-1 * max_point)
            eit_max_points_time[i].append(time() - start_time)
        eit_optimizer.register(params=next_point_to_probe, target=result,delta_t=delta_t)
        output[num_points_tested].append([num_points_tested, time() - start_time, result, cost_to_evaluate_eit, delta_t, was_best])
    print(f"EIT Total Points Tested: {num_points_tested}")
    offset = num_points_tested + 1
    output.append(["EI:"])
    ei_optimizer = BayesianOptimization(
        f=current_optimizing_fun,
        pbounds=bounds,
        allow_duplicate_points=True,
        time_travel=True
    )

    utility = UtilityFunction(kind="ei", kappa=2.5, xi=0.01)
    max_point = -1 * np.inf

    start_time = time()
    num_points_tested = 0
    while time() - start_time < 1800:
        output.append([])
        num_points_tested += 1
        earliest_t = time()
        next_point_to_probe = ei_optimizer.suggest(utility)
        print(next_point_to_probe)
        before_t = time()
        cost_to_evaluate_eit = before_t - earliest_t
        result = current_optimizing_fun(**next_point_to_probe)
        delta_t = time() - before_t
        was_best = False
        if result > max_point:
            was_best = True
            max_point = result
            ei_max_points[i].append(-1 * max_point)
            ei_max_points_time[i].append(time() - start_time)
            #print(f"Maximum Found: {max_point}")
        ei_optimizer.register(params=next_point_to_probe, target=result,delta_t=delta_t)
        output[num_points_tested + offset].append([num_points_tested, time() - start_time, result, cost_to_evaluate_eit, delta_t,was_best])
    print(f" EI Total Points Tested: {num_points_tested}")

with open('output', 'w') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerows(output)

